In [ ]:
# Import required libraries
import pandas as pd
#import dash
#import dash_html_components as html
#import dash_core_components as dcc
#from dash.dependencies import Input, Output
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_sites = spacex_df['Launch Site'].value_counts().index.tolist()

options_launch =  [{'label': 'All Sites', 'value': 'ALL'}]
for site in launch_sites:
    options_launch.append({'label':site,'value':site})
# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div([html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                                options=options_launch,
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                # Function decorator to specify function input and output
                                
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={i:str(i) for i in range(0,11000,1000)},
                                    value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output

# return the outcomes piechart for a selected site

@app.callback(
    Output(component_id='success-pie-chart',        component_property='figure'
    ),
    Input(component_id='site-dropdown', component_property='value')
)

def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        fig = px.pie(values=spacex_df[['Launch Site','class']].groupby('Launch Site').mean()['class'].tolist(), 
        names=spacex_df[['Launch Site','class']].groupby('Launch Site',as_index=False).mean()['Launch Site'].tolist(), 
        title='Success Rate')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.pie(values=filtered_df['class'].value_counts(normalize=True).tolist(), 
        names=['0','1'], 
        title='Success Rate')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart',        component_property='figure'
    ),
    Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")
)
def get_scatter_chart(site_dropdown,payload_slider):
    mask_dropdown = spacex_df['Launch Site'] == site_dropdown
    source_df = spacex_df.query(f"`Payload Mass (kg)`>{payload_slider[0]} and `Payload Mass (kg)`<{payload_slider[1]}")
    print(payload_slider[1],type(payload_slider[1]))
    filtered_df = source_df[mask_dropdown]
    
    if site_dropdown == 'ALL':
        fig = px.scatter(source_df,x='Payload Mass (kg)',y='class', 
        title='Payload vs Launch Result',color='Booster Version Category')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.scatter(filtered_df,x='Payload Mass (kg)',y='class', 
        title='Payload vs Launch Result',color='Booster Version Category')
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
